In [1]:
import torch, numpy as np, pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)

In [2]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.16-py3-none-any.whl size=110685 sha256=d24076cc8989fd7c32cb3c51f230a536fa675ad2e30dd9679c201a80dfdf8152
  Stored in directory: /root/.cache/pip/wheels/d2/ed/a5/da3a0cfb13373d1ace41cafa4f2467d858c55c52473ba72799
Successfully built kaggle


In [3]:
# !cp ../kaggle.json /root/.kaggle/kaggle.json


In [4]:
# Copying over from JH notebook
import os
from pathlib import Path
import pandas as pd
import numpy as np

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle: path = Path('../input/titanic')
else:
    path = Path('titanic')
    if not path.exists():
        import zipfile,kaggle
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

In [5]:
import pandas as pd
import numpy as np
import copy
orig_train_df = pd.read_csv(path/'train.csv')
train_df = copy.deepcopy(orig_train_df)

In [6]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [7]:
# TODO: Check which ones JH included
cat_vars = ['Pclass', 'Sex', 'Embarked']
num_vars = ['Age', 'SibSp', 'Parch', 'Fare']

In [8]:
num_df = train_df[num_vars].fillna(train_df[num_vars].mode().iloc[0])
num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)

# Going through an easier approach rather than typical norm
num_df = num_df / num_df.max()
num_df

,Age,SibSp,Parch,Fare
0,0.2750,0.125,0.000000,0.317521
1,0.4750,0.125,0.000000,0.683873
2,0.3250,0.000,0.000000,0.331789
3,0.4375,0.125,0.000000,0.636672
4,0.4375,0.000,0.000000,0.334298
...,...,...,...,...
886,0.3375,0.000,0.000000,0.411118
887,0.2375,0.000,0.000000,0.545154
888,0.3000,0.125,0.333333,0.505672
889,0.3250,0.000,0.000000,0.545154


In [9]:
train_df[cat_vars].describe(include='all')
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [10]:
train_df.Pclass.unique()

array([3, 1, 2])

In [11]:
# Curious how in-depth you should conside N/As
cat_df = pd.get_dummies(train_df[cat_vars], columns=cat_vars)
cat_df

,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1,0,1,0,0,1
1,1,0,0,1,0,1,0,0
2,0,0,1,1,0,0,0,1
3,1,0,0,1,0,0,0,1
4,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...
886,0,1,0,0,1,0,0,1
887,1,0,0,1,0,0,0,1
888,0,0,1,1,0,0,0,1
889,1,0,0,0,1,1,0,0


In [12]:
# Might need to normalize after creating the dummy vars?
proc_train_df = num_df.join(cat_df)

In [13]:
# Realize, going back to this, that we're not saving the mode but taking it for each piece of data we bring in
def process_df(path, has_dependent=True):
    df = pd.read_csv(path)
    df = df.fillna(df.mode().iloc[0])
    
    
    # TODO: Check which ones JH included
    cat_vars = ['Pclass', 'Sex', 'Embarked']
    num_vars = ['Age', 'SibSp', 'Parch', 'Fare']
    
    
    num_df = df[num_vars]
    num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)
    # Going for an easier approach rather than typical norm
    num_df = num_df / num_df.max()

    cat_df = pd.get_dummies(df[cat_vars], columns=cat_vars)
    cat_df['const'] = 1
    
    if has_dependent:
        return (num_df.join(cat_df), df['Survived'])
    else:
        return num_df.join(cat_df)

In [14]:
train_df, train_dep = process_df(path/'train.csv')
t_dep = torch.tensor(train_dep)
train_df

/tmp/ipykernel_84/3771005558.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)


,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,const
0,0.2750,0.125,0.000000,0.317521,0,0,1,0,1,0,0,1,1
1,0.4750,0.125,0.000000,0.683873,1,0,0,1,0,1,0,0,1
2,0.3250,0.000,0.000000,0.331789,0,0,1,1,0,0,0,1,1
3,0.4375,0.125,0.000000,0.636672,1,0,0,1,0,0,0,1,1
4,0.4375,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.3375,0.000,0.000000,0.411118,0,1,0,0,1,0,0,1,1
887,0.2375,0.000,0.000000,0.545154,1,0,0,1,0,0,0,1,1
888,0.3000,0.125,0.333333,0.505672,0,0,1,1,0,0,0,1,1
889,0.3250,0.000,0.000000,0.545154,1,0,0,0,1,1,0,0,1


In [15]:
train_df.isna().sum()

Age           0
SibSp         0
Parch         0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
const         0
dtype: int64

In [16]:
test_df = process_df(path/'test.csv', has_dependent=False)
test_df

/tmp/ipykernel_84/3771005558.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)


,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,const
0,0.453947,0.000,0.000000,0.329840,0,0,1,0,1,0,1,0,1
1,0.618421,0.125,0.000000,0.311896,0,0,1,1,0,0,0,1,1
2,0.815789,0.000,0.000000,0.363976,0,1,0,0,1,0,1,0,1
3,0.355263,0.000,0.000000,0.346051,0,0,1,0,1,0,0,1,1
4,0.289474,0.125,0.111111,0.402083,0,0,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.276316,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1
414,0.513158,0.000,0.000000,0.751796,1,0,0,1,0,1,0,0,1
415,0.506579,0.000,0.000000,0.317521,0,0,1,0,1,0,0,1,1
416,0.276316,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1


In [98]:
tst_indep = torch.tensor(test_df)
tst_indep

ValueError: could not determine the shape of object type 'DataFrame'

In [97]:
test_df.isna().sum()

Age           0
SibSp         0
Parch         0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
const         0
dtype: int64

In [18]:
# Great success! We can move into building out model!

In [19]:
import torch

In [20]:
weights = torch.rand(train_df.shape[1]) - 0.5
weights

tensor([-0.2356, -0.3660, -0.3669,  0.1891, -0.1264,  0.2338, -0.1827,  0.0144, -0.3984,  0.4174,  0.4224, -0.1842,  0.1439])

In [21]:
tensor_df = torch.tensor(train_df.values)
tensor_df

tensor([[0.2750, 0.1250, 0.0000, 0.3175, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4750, 0.1250, 0.0000, 0.6839, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000],
        [0.3250, 0.0000, 0.0000, 0.3318, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4375, 0.1250, 0.0000, 0.6367, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4375, 0.0000, 0.0000, 0.3343, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.3000, 0.0000, 0.0000, 0.3422, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000],
        [0.6750, 0.0000, 0.0000, 0.6329, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        ...,
        [0.3125, 0.0000, 0.0000, 0.3130, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4875, 0.0000, 0.8333, 0.5404, 0.0000, 0.0000, 1.0000, 1.0000, 0.

In [22]:
result = (tensor_df * weights)
result

tensor([[-0.0648, -0.0457, -0.0000,  0.0601, -0.0000,  0.0000, -0.1827,  0.0000, -0.3984,  0.0000,  0.0000, -0.1842,  0.1439],
        [-0.1119, -0.0457, -0.0000,  0.1293, -0.1264,  0.0000, -0.0000,  0.0144, -0.0000,  0.4174,  0.0000, -0.0000,  0.1439],
        [-0.0766, -0.0000, -0.0000,  0.0628, -0.0000,  0.0000, -0.1827,  0.0144, -0.0000,  0.0000,  0.0000, -0.1842,  0.1439],
        [-0.1031, -0.0457, -0.0000,  0.1204, -0.1264,  0.0000, -0.0000,  0.0144, -0.0000,  0.0000,  0.0000, -0.1842,  0.1439],
        [-0.1031, -0.0000, -0.0000,  0.0632, -0.0000,  0.0000, -0.1827,  0.0000, -0.3984,  0.0000,  0.0000, -0.1842,  0.1439],
        [-0.0707, -0.0000, -0.0000,  0.0647, -0.0000,  0.0000, -0.1827,  0.0000, -0.3984,  0.0000,  0.4224, -0.0000,  0.1439],
        [-0.1590, -0.0000, -0.0000,  0.1197, -0.1264,  0.0000, -0.0000,  0.0000, -0.3984,  0.0000,  0.0000, -0.1842,  0.1439],
        ...,
        [-0.0736, -0.0000, -0.0000,  0.0592, -0.0000,  0.0000, -0.1827,  0.0000, -0.3984,  0.0000,

In [23]:
result.shape

torch.Size([891, 13])

In [24]:
train_df.shape

(891, 13)

In [25]:
def sigmoid(x):
    return 1 / (1 + 1 / torch.exp(-x))

In [26]:
torch_implementation_sigmoid = torch.sigmoid(result.sum(axis=1))
my_implementation_sigmoid = sigmoid(result.sum(axis=1))

In [27]:
# Awesome!
(torch_implementation_sigmoid == torch_implementation_sigmoid).sum()

tensor(891)

In [28]:
weights

tensor([-0.2356, -0.3660, -0.3669,  0.1891, -0.1264,  0.2338, -0.1827,  0.0144, -0.3984,  0.4174,  0.4224, -0.1842,  0.1439])

In [29]:
# TODO: Implement actual training

import torch
torch.manual_seed(0)
import random
random.seed(0)
import numpy as np
np.random.seed(0)

def mae(preds, actual):
    return torch.abs(preds - actual).mean()

actual = torch.tensor(orig_train_df['Survived'])
weights = torch.rand(train_df.shape[1]) - 0.5
weights.requires_grad_()

def training_step(step, lr):
    global weights
    
    layer_results = tensor_df * weights
    preds = sigmoid(layer_results.sum(axis=1))
    loss = mae(preds, actual)

    
    if step % 100 == 0:
        print("loss:", loss)

    # print(weights.data)
        
    # if torch.isnan(loss):
    #     print(weights)
        
    loss.backward()
    
    with torch.no_grad():
        weights -= lr * weights.grad
        
def train(steps=10000, lr=3e-3):
    for i in range(steps):
        training_step(i, lr)
        
def pred(input_tensor):
    layer_results = input_tensor * weights
    return sigmoid(layer_results.sum(axis=1))

train()

loss: tensor(0.5500, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.3298, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2631, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2053, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2049, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2095, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2113, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2120, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2123, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2125, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2126, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2126, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2125, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2124, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2122, dtype=torch.float64, grad_fn=<MeanBackwar

In [30]:
weights.data

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [31]:
weights - weights.grad

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], grad_fn=<SubBackward0>)

In [32]:
# Since weights are blowing up, going back through and checking my notebook against JH
# TODO: Plug in fastbook to write these up; learn in public bb
train_df

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,const
0,0.2750,0.125,0.000000,0.317521,0,0,1,0,1,0,0,1,1
1,0.4750,0.125,0.000000,0.683873,1,0,0,1,0,1,0,0,1
2,0.3250,0.000,0.000000,0.331789,0,0,1,1,0,0,0,1,1
3,0.4375,0.125,0.000000,0.636672,1,0,0,1,0,0,0,1,1
4,0.4375,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.3375,0.000,0.000000,0.411118,0,1,0,0,1,0,0,1,1
887,0.2375,0.000,0.000000,0.545154,1,0,0,1,0,0,0,1,1
888,0.3000,0.125,0.333333,0.505672,0,0,1,1,0,0,0,1,1
889,0.3250,0.000,0.000000,0.545154,1,0,0,0,1,1,0,0,1


## Exploring NaN weights

- Like the options to change display:

```python
import torch, numpy as np, pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)
```

- Doesn't look like he "normalized" numerical values
- cat_vars = [sex, pclass, embarked] which lines up with mine!
- num_vars line up as well, Age, SibSp, Parch, and Fare.
- Primary difference looks like he didn't
- Also his naming scheme looks a lot cleaner than mine; something to learn here
- I added a const column, might be better, but JH might've been just going the easy way

- Ahhhhh looks like problem might be `coeffs.grad.zero_()`

In [33]:
weights = torch.rand(train_df.shape[1]) - 0.5
tensor_df * weights

tensor([[-0.0911, -0.0258,  0.0000,  0.0628,  0.0000, -0.0000, -0.2177,  0.0000,  0.4152, -0.0000,  0.0000, -0.0806,  0.0529],
        [-0.1573, -0.0258,  0.0000,  0.1352,  0.3000, -0.0000, -0.0000,  0.1816,  0.0000, -0.1029,  0.0000, -0.0000,  0.0529],
        [-0.1076, -0.0000,  0.0000,  0.0656,  0.0000, -0.0000, -0.2177,  0.1816,  0.0000, -0.0000,  0.0000, -0.0806,  0.0529],
        [-0.1449, -0.0258,  0.0000,  0.1258,  0.3000, -0.0000, -0.0000,  0.1816,  0.0000, -0.0000,  0.0000, -0.0806,  0.0529],
        [-0.1449, -0.0000,  0.0000,  0.0661,  0.0000, -0.0000, -0.2177,  0.0000,  0.4152, -0.0000,  0.0000, -0.0806,  0.0529],
        [-0.0993, -0.0000,  0.0000,  0.0676,  0.0000, -0.0000, -0.2177,  0.0000,  0.4152, -0.0000,  0.3742, -0.0000,  0.0529],
        [-0.2235, -0.0000,  0.0000,  0.1251,  0.3000, -0.0000, -0.0000,  0.0000,  0.4152, -0.0000,  0.0000, -0.0806,  0.0529],
        ...,
        [-0.1035, -0.0000,  0.0000,  0.0619,  0.0000, -0.0000, -0.2177,  0.0000,  0.4152, -0.0000,

In [34]:
tensor_df.shape, weights.shape

(torch.Size([891, 13]), torch.Size([13]))

In [35]:
tensor_df

tensor([[0.2750, 0.1250, 0.0000, 0.3175, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4750, 0.1250, 0.0000, 0.6839, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000],
        [0.3250, 0.0000, 0.0000, 0.3318, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4375, 0.1250, 0.0000, 0.6367, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4375, 0.0000, 0.0000, 0.3343, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.3000, 0.0000, 0.0000, 0.3422, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000],
        [0.6750, 0.0000, 0.0000, 0.6329, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        ...,
        [0.3125, 0.0000, 0.0000, 0.3130, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4875, 0.0000, 0.8333, 0.5404, 0.0000, 0.0000, 1.0000, 1.0000, 0.

In [36]:
weights.shape

torch.Size([13])

In [37]:
!git config --global user.email "treyroyalt@gmail.com"
!git config --global user.name "jpr2000"
!git commit -m "Fix training with validation stats"

In [38]:
preds = (tensor_df*weights).sum(axis=1)
preds[:10]

tensor([ 0.1157,  0.3838, -0.1058,  0.4091,  0.0910,  0.5928,  0.5891,  0.1839, -0.0930, -0.1832], dtype=torch.float64)

In [39]:
loss = torch.abs(t_dep - preds).mean()
loss

tensor(0.4450, dtype=torch.float64)

In [40]:
def calc_preds(inputs, weights): return sigmoid((inputs*weights).sum(axis=1))
def calc_loss(inputs, weights, actual): 
    return torch.abs(actual - calc_preds(inputs, weights)).mean()

In [41]:
calc_loss(tensor_df, weights, t_dep)

tensor(0.4949, dtype=torch.float64)

In [42]:
weights

tensor([-0.3311, -0.2061,  0.0185,  0.1977,  0.3000, -0.3390, -0.2177,  0.1816,  0.4152, -0.1029,  0.3742, -0.0806,  0.0529])

In [43]:
# Going to see where we went wrong

import torch
import random
import numpy as np

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

weights = torch.rand(train_df.shape[1]) - 0.5
weights.requires_grad_()

def training_step(step, lr):
    global weights
    
    # layer_results = tensor_df * weights
    # preds = sigmoid(layer_results.sum(axis=1))
    # loss = mae(preds, actual)
    
    loss = calc_loss(tensor_df, weights, t_dep)

    # print(weights.data)
        
    # if torch.isnan(loss):
    #     print(weights)
        
    loss.backward()
    
    with torch.no_grad():
        weights.sub_(weights.grad * lr)
        
        # Think the following line is where we went wrong
        weights.grad.zero_()
        if step % 100 == 0:
            print("loss:", loss)
            print("val loss:", )
        
        
def train(steps=10000, lr=0.1):
    global weights
    weights = torch.rand(train_df.shape[1]) - 0.5
    weights.requires_grad_()
    
    for i in range(steps):
        training_step(i, lr)
        
train()

loss: tensor(0.4949, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.3642, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.3171, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2867, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2683, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2567, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2488, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2431, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2388, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2354, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2327, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2304, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2286, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:

Results now look similar to Jeremy's! Importance of zeroing out your gradients!

In [44]:
weights

tensor([ 0.2976,  1.1950,  1.0525, -0.3461, -1.6866, -1.7561,  3.2334, -4.5529,  5.1971, -0.7122, -1.3137,  2.2640,  0.1003],
       requires_grad=True)

In [ ]:
train(lr=1e2)

## Let's actually build out a validation set

In [46]:
from fastai.data.transforms import RandomSplitter

train_idx, val_idx = RandomSplitter(seed=42)(train_df)

trn_indep, val_indep = tensor_df[train_idx], tensor_df[val_idx]
trn_dep, val_dep     = t_dep[train_idx], t_dep[val_idx]

In [47]:
# Verifying we still have all our data 
assert trn_indep.shape[0] + val_indep.shape[0] == 891

In [57]:
??calc_loss

Signature: calc_loss(inputs, weights, actual)
Docstring: <no docstring>
Source:   
def calc_loss(inputs, weights, actual): 
    return torch.abs(actual - calc_preds(inputs, weights)).mean()
File:      /tmp/ipykernel_84/3909039809.py
Type:      function


In [92]:
# Updating for accuracy
# TODO: Need a better naming scheme

# TODO: Update this with new variables

import torch
import random
import numpy as np

weights

def training_step(step, lr):
    global weights
    
    loss = calc_loss(trn_indep, weights, trn_dep)
    loss.backward()
    
    with torch.no_grad():
        weights.sub_(weights.grad * lr)
        weights.grad.zero_()
        
        if step % 1000 == 0:
            trn_loss = calc_loss(trn_indep, weights, trn_dep).item()
            val_loss = calc_loss(val_indep, weights, val_dep).item()
            print(trn_loss, val_loss)
        
        
def train(steps=10000, lr=0.1):
    torch.manual_seed(0)
    random.seed(0)
    np.random.seed(0)
    
    global weights
    weights = torch.rand(train_df.shape[1]) - 0.5
    weights.requires_grad_()
    
    for i in range(steps):
        training_step(i, lr)

In [96]:
train(lr=1e0)

0.5303622207049395 0.5319113867953892
0.20148153446224695 0.18740216158440876
0.19735286147191075 0.18160683233195013
0.19579616599382862 0.17934296847835493
0.1949595229515259 0.17811480634403976
0.19443323892736752 0.1773427211462587
0.19407070990278433 0.1768129781311443
0.19380553766900835 0.17642739246372777
0.1936030634730265 0.17613443709475138
0.19344336782294858 0.17590446493743944


In [51]:
??calc_loss

Signature: calc_loss(inputs, weights, actual)
Docstring: <no docstring>
Source:   
def calc_loss(inputs, weights, actual): 
    return torch.abs(actual - calc_preds(inputs, weights)).mean()
File:      /tmp/ipykernel_84/3909039809.py
Type:      function


In [49]:
# Verifying we still have all our data 
train_indep.shape[0] + val_indep.shape[0]

NameError: name 'train_indep' is not defined